In [1]:
import numpy as np
import tensorflow as tf
import chess
import pandas as pd
import io

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, Lambda
from keras import backend as K
from tensorflow import keras

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using TensorFlow backend.


Helper Functions~~~

In [0]:
NUM_OUTPUTS = 5


def get_mat(board, piece, color):
  squareSet = board.pieces(piece, color)
  return get_bin_mat(squareSet);

def get_bin_mat(squareSet):
  mat = np.zeros((8,8))
  for x in range(0, 8):
    for y in range(0, 8):
        if(y*8 + x in squareSet):
          # The 7 - is used to flip the board representation
          mat[7 - y, x] = 1
            
  return mat


def get_dual_bin_mat(piece):
  whiteSquareSet = board.pieces(piece, chess.WHITE)
  blackSquareSet = board.pieces(piece, chess.BLACK)
  mat = np.zeros((8,8))
  for x in range(0, 8):
    for y in range(0, 8):
        if(y*8 + x in whiteSquareSet):
          # The 7 - is used to flip the board representation
          mat[7 - y, x] = 1
        elif(y*8 + x in blackSquareSet):
          mat[7 - y, x] = -1
            
  return mat


def interpret_pred(p):
  num_outputs = len(p)
  
  moves = []
  for out in p:
    # There's an extra list wrapping for some reason
    out = out[0]
    start = np.argmax(out[:64])
    end = np.argmax(out[64:])
       
    moves.append((chess.SQUARE_NAMES[start],
                  chess.SQUARE_NAMES[end]))
    
  return moves

def create_input(board):
  mat = np.array(([get_mat(board, i, chess.WHITE) for i in range(1,7)] + [get_mat(board, i, chess.BLACK) for i in range(1,7)]))
  y = mat.reshape((-1, 12, 8, 8))
  return y

def create_output(moves):
  output = []
  for i in range(0, NUM_OUTPUTS):
    startPos = moves[i][0]
    endPos = moves[i][1]
    
    mat = np.zeros(128)
    
    mat[startPos] = 1
    mat[64 + endPos] = 1
        
    output.append(mat)
  
  return output

In [3]:
np.zeros(128).shape

(128,)

Input data:

  (6 pieces + 1 for meta state ) * 2 players
  
  8 rows
  
  8 columns
  
 => 8 * 8 * 12 = 768
 
 
 Output data:
 
 - Assume predicting five moves
 - initial location, and new location
 
 
Array of objects/tuples
[{initial location, new location},...]
 
 
 => 5 * 2 * 64 = 640
  

In [9]:
inp = Input((12,8,8))


x = Conv2D(64, kernel_size=3, activation='relu', input_shape=(12,8,8))(inp)

x = Conv2D(64, kernel_size=3, activation='relu', input_shape=(12,8,8))(x)

x = Conv2D(32, kernel_size=3, activation='relu', input_shape=(12,8,8))(x)


x = Flatten()(x)


outputs = []

for n in range(NUM_OUTPUTS):
  outputs.append(Dense(128, activation='softmax')(x))

model = Model(inp, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 12, 8, 8)]   0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 10, 6, 64)    4672        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 8, 4, 64)     36928       conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 6, 2, 32)     18464       conv2d_4[0][0]                   
____________________________________________________________________________________________

In [5]:
board = chess.Board()

y = create_input(board)

p = model.predict(y)

interpret_pred(p)

[('d6', 'g5'), ('b5', 'a7'), ('a1', 'h4'), ('b5', 'f7'), ('h8', 'g8')]

In [6]:
df=pd.read_csv('drive/My Drive/Chess/train.csv')

test_train = df.head()
test_train

,Position,Moves
0,rnbqkbnr/ppppppp1/7p/8/4P3/2N5/PPPP1PPP/R1BQKB...,"[(62, 45), (52, 36), (49, 41), (51, 43), (50, ..."
1,rnbqkbnr/pp1pppp1/7p/2p5/4P3/2N5/PPPP1PPP/R1BQ...,"[(18, 1), (18, 33), (5, 33), (13, 29), (0, 1)]"
2,rnbqkbnr/pp1pppp1/7p/2p5/4P3/8/PPPP1PPP/RNBQKB...,"[(54, 38), (53, 45), (57, 42), (52, 36), (59, ..."
3,rnbqkbnr/pp1ppp2/7p/2p3p1/4P3/8/PPPP1PPP/RNBQK...,"[(6, 23), (10, 26), (3, 12), (10, 18), (28, 36)]"
4,rnbqkbnr/pp1ppp2/7p/2p3p1/4P3/3B4/PPPP1PPP/RNB...,"[(48, 32), (57, 42), (59, 32), (34, 26), (59, ..."


In [0]:
inputs = []
outputs = []

for i in range(len(test_train)):
  pos = test_train.iloc[i].Position
  moves = eval(test_train.iloc[i].Moves)
  
  board.set_fen(pos)
  if i > 0:
    inputs = np.append(inputs, create_input(board), axis=0)
  else:
    inputs = create_input(board)
  outputs.append(create_output(moves))


In [8]:
model.fit(inputs, outputs)

5/5 [==============================] - 0s 48ms/sample - loss: 48.5043 - dense_loss: 9.6973 - dense_1_loss: 9.6651 - dense_2_loss: 9.7240 - dense_3_loss: 9.6857 - dense_4_loss: 9.7322 - dense_acc: 0.0000e+00 - dense_1_acc: 0.0000e+00 - dense_2_acc: 0.0000e+00 - dense_3_acc: 0.0000e+00 - dense_4_acc: 0.0000e+00
